### Load Libraries

In [1]:
import pandas as pd
import numpy as np
import os
os.getcwd()

'/Users/deepakm/mywork/mynlp/txnstmt'

### Load the Data

In [3]:
dtxn=pd.read_csv('flat_transactions.csv')
dtxn.head()

,kde_id,id,country,description,transactionDate,transactionAmount,transactionAmountCurrency,bankTransactionFlowType,currentBalanceAmount,currentBalanceCurrency,accountIdentifier,bankIdentifier,amountInFunctionalCurrencyAmount,amountInFunctionalCurrencyCurrency,currentBalanceInFunctionalCurrencyAmount,currentBalanceInFunctionalCurrencyCurrency,bankDataSource
0,10048,1,NL,AUTOMAT S6EE6080 0063 Karte3 27.06.2015 UM 18:...,2015-06-29,-400.00,EUR,OUTFLOW,-6006.540039,EUR,170331,Bawag,-400.00,EUR,-6006.540039,EUR,titanium-db
1,10048,2,NL,---- 0103619245 / 200003507629 ----Rechnungsnr...,2015-06-29,-38.08,EUR,OUTFLOW,-6406.540039,EUR,170331,Bawag,-38.08,EUR,-6406.540039,EUR,titanium-db
2,10048,257,NL,150351,2015-06-29,228.64,EUR,INFLOW,-48527.500000,EUR,170511,Bawag,228.64,EUR,-48527.500000,EUR,titanium-db
3,10048,258,NL,"Kundennummer 000000086189, Hutchison Drei Aust...",2015-06-29,-30.22,EUR,OUTFLOW,-48298.859380,EUR,170511,Bawag,-30.22,EUR,-48298.859380,EUR,titanium-db
4,10048,259,NL,---- 0103619245 / 200003768285 ----Rechnungsnr...,2015-06-29,-34.98,EUR,OUTFLOW,-48329.078130,EUR,170511,Bawag,-34.98,EUR,-48329.078130,EUR,titanium-db


In [4]:
dtoutflow=dtxn.loc[dtxn.bankTransactionFlowType=='OUTFLOW']
dtinflow=dtxn.loc[dtxn.bankTransactionFlowType=='INFLOW']

In [5]:
dtoutflow.head()

,kde_id,id,country,description,transactionDate,transactionAmount,transactionAmountCurrency,bankTransactionFlowType,currentBalanceAmount,currentBalanceCurrency,accountIdentifier,bankIdentifier,amountInFunctionalCurrencyAmount,amountInFunctionalCurrencyCurrency,currentBalanceInFunctionalCurrencyAmount,currentBalanceInFunctionalCurrencyCurrency,bankDataSource
0,10048,1,NL,AUTOMAT S6EE6080 0063 Karte3 27.06.2015 UM 18:...,2015-06-29,-400.00,EUR,OUTFLOW,-6006.540039,EUR,170331,Bawag,-400.00,EUR,-6006.540039,EUR,titanium-db
1,10048,2,NL,---- 0103619245 / 200003507629 ----Rechnungsnr...,2015-06-29,-38.08,EUR,OUTFLOW,-6406.540039,EUR,170331,Bawag,-38.08,EUR,-6406.540039,EUR,titanium-db
3,10048,258,NL,"Kundennummer 000000086189, Hutchison Drei Aust...",2015-06-29,-30.22,EUR,OUTFLOW,-48298.859380,EUR,170511,Bawag,-30.22,EUR,-48298.859380,EUR,titanium-db
4,10048,259,NL,---- 0103619245 / 200003768285 ----Rechnungsnr...,2015-06-29,-34.98,EUR,OUTFLOW,-48329.078130,EUR,170511,Bawag,-34.98,EUR,-48329.078130,EUR,titanium-db
11,10048,266,NL,Kd.Nr:234545 Re.Nr:151119,2015-06-30,-378.42,EUR,OUTFLOW,-47729.261720,EUR,170511,Bawag,-378.42,EUR,-47729.261720,EUR,titanium-db


### Text Data as a List for Embeddings

In [6]:
txtlst=dtxn.loc[dtxn.bankTransactionFlowType=='OUTFLOW','description'].tolist()
# txtlst=dtxn.loc[dtxn.bankTransactionFlowType=='INFLOW','description'].tolist()

In [7]:
print("Number of Text Records: ",len(txtlst))

Number of Text Records:  294120


In [8]:
# For Testing take 1,000 records
txtlst=txtlst[:1000]

### Vectorization of text
Three different methods for Vectorization

#### TFIDF Vectorization

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
n_features = 1000
vectorizer = TfidfVectorizer(max_df=0.5, max_features=n_features,min_df=2, stop_words='english')
embeddings = vectorizer.fit_transform(txtlst)

#### Google's Universal Sentence Encoder Embeddings

In [ ]:
# !pip install tensorflow_hub

In [12]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
# embed=hub.load("pretrained_models/universal-sentence-encoder_4")
embeddings=embed(txtlst)

#### BERT Model Embeddings
List of Available pre-trained models
https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/edit#gid=0

In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
embeddings = model.encode(txtlst, show_progress_bar=True)

In [9]:
embeddings.shape

(1000, 768)

### Reduce the dimensions of Embeddings 
Results in Better Clustering for HDBSCAN

In [10]:
from umap import UMAP
embeddings_reduced = UMAP(n_neighbors=10,n_components=10, metric='cosine').fit_transform(embeddings)

In [11]:
embeddings_reduced.shape

(1000, 10)

### Normalize the Embeddings

In [12]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer(copy=False)
embeddings_reduced = normalizer.fit_transform(embeddings_reduced)
print("Shape of the Embeddigns: ",embeddings_reduced.shape)
embeddings_reduced

Shape of the Embeddigns:  (1000, 10)


array([[ 0.505014  ,  0.33204058,  0.2743354 , ...,  0.43222037,
         0.02396787,  0.16670649],
       [ 0.4405385 ,  0.4219402 ,  0.29253492, ...,  0.38031182,
         0.34338686,  0.28933713],
       [ 0.40565377,  0.35750765,  0.3374276 , ...,  0.31277385,
         0.25131115,  0.31434304],
       ...,
       [-0.21629553,  0.91166437,  0.06125591, ...,  0.05537823,
         0.05344417,  0.04802498],
       [-0.18295628,  0.10058299,  0.84984106, ...,  0.14855747,
         0.1737281 ,  0.18422018],
       [-0.18589461,  0.10145242,  0.8501376 , ...,  0.1503816 ,
         0.17330368,  0.18461928]], dtype=float32)

In [ ]:
# embeddings = normalizer.fit_transform(embeddings)
# print("Shape of the Embeddigns: ",embeddings.shape)
# embeddings

### Clustering using Embeddings
Methods: HDBSCAN and KMeans

Cluster on the Embeddings or Reduced Embeddings

In [13]:
# Cluster on the Full Embeddings
# X = embeddings

# Cluster on the Reduced Embeddings
X = embeddings_reduced

print("Shape of Input: ",X.shape)

Shape of Input:  (1000, 10)


#### HDBSCAN Algorithm

In [14]:
from hdbscan import HDBSCAN
clusterer = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom').fit(X)

cluster_assignment = clusterer.labels_

num_clus=clusterer.labels_.max()

print("Number of Clusters found: ",num_clus)
# print("Cluster Probabilities: ",clusterer.probabilities_)

Number of Clusters found:  42


In [15]:
cluster_assignment.shape

(1000,)

#### KMeans Clustering

Find Optimal Number of Cluster

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

#METHOD 1: Elbow curve based on SSE 

sse = {}
slh={}
for k in range(2, 30):
    kmeans = KMeans(n_clusters=k, max_iter=100).fit(X)
    sse[k] = kmeans.inertia_ 
    label  = kmeans.labels_
    slh[k] = silhouette_score(X, label, metric='euclidean')
#PLOT THE GRAPHS
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5))
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
# plt.show()
#METHOD 2: Using  silhouette score: higher the better
plt.figure(figsize=(15, 5))
plt.plot(list(slh.keys()), list(slh.values()))
plt.xlabel("Number of cluster")
plt.ylabel("Silhouette Score")
# plt.show()

In [ ]:
num_clus=max(slh,key=slh.get)
print("Optimal Cluster for KMeans: ",num_clus)

# IF NUMBER OF CLUSTERS ARE VERY HIGH LIMIT IT TO 20 CLUSTERS
num_clus = min(20,num_clus)
print("Number of Clusters for KMeans: ",num_clus)

Cluster on the Embeddings or Reduced Embeddings

In [ ]:
num_clus = num_clus
# num_clus=20
clusterer = KMeans(n_clusters=num_clus, init='k-means++', max_iter=100, n_init=2,random_state=123)
clusterer.fit(X)
cluster_assignment = clusterer.labels_

In [ ]:
cluster_assignment.shape

### GET TOP N WORDS BY CLUSTER
The text is aggregated by cluster and then tfidf weighted most common words are found for each cluster

In [16]:
docs_df = pd.DataFrame(txtlst, columns=["Doc"])
docs_df['Clust'] = clusterer.labels_
docs_df['Doc_ID'] = range(len(docs_df))
print(docs_df.head())
docs_per_cluster = docs_df.groupby(['Clust'], as_index = False).agg({'Doc': ' '.join})
docs_per_cluster.head()

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count


def extract_top_n_words_per_cluster(tf_idf, count, docs_per_cluster, n=10):
    words = count.get_feature_names()
    labels = list(docs_per_cluster.Clust)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_cluster_sizes(df):
    cluster_sizes = (df.groupby(['Clust'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Clust": "Clust", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return cluster_sizes

tf_idf, count = c_tf_idf(docs_per_cluster.Doc.values, m=len(txtlst))
top_n_words = extract_top_n_words_per_cluster(tf_idf, count, docs_per_cluster, n=10)
# cluster_sizes = extract_cluster_sizes(docs_df)
# print("Number of Clusters Found: ",cluster_sizes.shape[0])
# cluster_sizes
print(top_n_words[1][:10])

topw={}
for k,v in top_n_words.items():
    topw[k]=[i[0] for i in v]

topw

                                                 Doc  Clust  Doc_ID
0  AUTOMAT S6EE6080 0063 Karte3 27.06.2015 UM 18:...     17       0
1  ---- 0103619245 / 200003507629 ----Rechnungsnr...     25       1
2  Kundennummer 000000086189, Hutchison Drei Aust...     33       2
3  ---- 0103619245 / 200003768285 ----Rechnungsnr...     25       3
4                        Kd.Nr:234545 Re.Nr:151119       30       4
[('hrung', 0.8415846231219198), ('entgelt', 0.6914971733648756), ('kontof', 0.6882596898665907), ('gen', 0.2846960320449817), ('manipulation', 0.2846960320449817), ('von', 0.2846960320449817), ('nichtdurchf', 0.2846960320449817), ('auftr', 0.2846960320449817), ('52834947', 0.0), ('52829560', 0.0)]


/Users/deepak/opt/anaconda3/envs/mynlp/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


{-1: ['x358853',
  'mbs',
  'bawag',
  'banking',
  'tze',
  'ums',
  'inkl',
  '000003',
  'teilnehmer',
  'vorschreibung'],
 0: ['zweitwohnsitzabg',
  '52829560',
  '52871620',
  '52871619',
  '52871617',
  '52871603',
  '52871602',
  '52871601',
  '52866203',
  '52834963'],
 1: ['hrung',
  'entgelt',
  'kontof',
  'gen',
  'manipulation',
  'von',
  'nichtdurchf',
  'auftr',
  '52834947',
  '52829560'],
 2: ['f006090',
  'zweitwohnsitzabg',
  '52834945',
  '52871620',
  '52871619',
  '52871617',
  '52871603',
  '52871602',
  '52871601',
  '52866203'],
 3: ['x731422',
  'x789056',
  'x041426',
  'zweitwohnsitzabg',
  '52834963',
  '52834945',
  '52834946',
  '52834947',
  '52834961',
  '52834962'],
 4: ['x046056',
  'x036200',
  'x635287',
  'zweitwohnsitzabg',
  '52834963',
  '52834946',
  '52834947',
  '52834961',
  '52834962',
  '52866203'],
 5: ['x358806',
  'i780650',
  'zweitwohnsitzabg',
  '52866203',
  '52834946',
  '52834947',
  '52834961',
  '52834962',
  '52834963',
  '528

### Assign the Clusters and Top Words in Original data

In [17]:
dtoutflow['cluster_assignment']=pd.Series(cluster_assignment)

dtoutflow['Top_Words_in_Cluster']=dtoutflow.cluster_assignment.map(topw)

dtoutflow.head()

/Users/deepak/opt/anaconda3/envs/mynlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,kde_id,id,country,description,transactionDate,transactionAmount,transactionAmountCurrency,bankTransactionFlowType,currentBalanceAmount,currentBalanceCurrency,accountIdentifier,bankIdentifier,amountInFunctionalCurrencyAmount,amountInFunctionalCurrencyCurrency,currentBalanceInFunctionalCurrencyAmount,currentBalanceInFunctionalCurrencyCurrency,bankDataSource,cluster_assignment
0,10048,1,NL,AUTOMAT S6EE6080 0063 Karte3 27.06.2015 UM 18:...,2015-06-29,-400.00,EUR,OUTFLOW,-6006.540039,EUR,170331,Bawag,-400.00,EUR,-6006.540039,EUR,titanium-db,17.0
1,10048,2,NL,---- 0103619245 / 200003507629 ----Rechnungsnr...,2015-06-29,-38.08,EUR,OUTFLOW,-6406.540039,EUR,170331,Bawag,-38.08,EUR,-6406.540039,EUR,titanium-db,25.0
3,10048,258,NL,"Kundennummer 000000086189, Hutchison Drei Aust...",2015-06-29,-30.22,EUR,OUTFLOW,-48298.859380,EUR,170511,Bawag,-30.22,EUR,-48298.859380,EUR,titanium-db,25.0
4,10048,259,NL,---- 0103619245 / 200003768285 ----Rechnungsnr...,2015-06-29,-34.98,EUR,OUTFLOW,-48329.078130,EUR,170511,Bawag,-34.98,EUR,-48329.078130,EUR,titanium-db,30.0
11,10048,266,NL,Kd.Nr:234545 Re.Nr:151119,2015-06-30,-378.42,EUR,OUTFLOW,-47729.261720,EUR,170511,Bawag,-378.42,EUR,-47729.261720,EUR,titanium-db,22.0


In [ ]:
###### IGNORE THIS ############################

# from sklearn.feature_extraction.text import TfidfVectorizer
# import numpy as np

# # docs_per_cluster.Doc.values
# def top_tfidf_words(docs,top_n=5):
#     try:
#         vectorizer = TfidfVectorizer()
#         X = vectorizer.fit_transform(docs)
#         indices = np.argsort(vectorizer.idf_)[::-1]
#         features = vectorizer.get_feature_names()
#         top_features = [features[i] for i in indices[:top_n]]
# #         print( top_features)
#         return list(top_features)
#     except ValueError:
#         return []

# docs_per_cluster['top_words']=docs_per_cluster.apply(lambda row: top_tfidf_words([row['Doc']],5),axis=1)

# docs_per_cluster.head(10)